In [15]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd


In [16]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=562 <class 'str'>


In [17]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [18]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [19]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1417


In [20]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1417


In [21]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7239541,562,NIR1469,1469,2020-10-03,2020,1,Riots,Mob violence,Rioters (Niger),...,Diffa,13.3200,12.6100,1,Whatsapp,New media,"On 3 October 2020, twenty-one prisoners (presu...",0,1601932936,NER
1,7239452,562,NIR1467,1467,2020-10-01,2020,1,Protests,Peaceful protest,Protesters (Niger),...,Niamey,13.5200,2.1200,1,Studio Kalangou,National,"On 1 October 2020, a number of teachers staged...",0,1601932936,NER
2,7239043,562,NIR1468,1468,2020-10-01,2020,1,Protests,Peaceful protest,Protesters (Niger),...,Zinder,13.6500,9.1833,1,Studio Kalangou; Actu (Niger),National,"On 1 October 2020, a number of university stud...",0,1601932935,NER
3,7239538,562,NIR1465,1465,2020-09-30,2020,1,Violence against civilians,Attack,Military Forces of Niger (2011-),...,Toumour,13.6669,13.1198,2,Whatsapp,New media,"On 30 September 2020, Nigerien soldiers report...",2,1601932936,NER
4,7239540,562,NIR1466,1466,2020-09-30,2020,1,Violence against civilians,Abduction/forced disappearance,Islamic State (West Africa),...,Zarwaram,13.1981,12.4835,1,Whatsapp,New media,"On 30 September 2020, ISWAP militants reported...",0,1601932936,NER


### Do the right thing, take some time to look at the codebook and see what these columns are

In [22]:
niger_res.columns


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [23]:
# Import additional libraries

from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Range1d
import math #this is used in graphic section to use the irrational number pi
output_notebook() #Allows inline plotting for Juptyer notebook

#Imports necessary aspects of Bokeh for plotting on a map
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
tile_provider = get_provider('STAMEN_TERRAIN')

Loading BokehJS ...

In [24]:
# niger_res.info()

In [69]:
# Take the data results from Niger and get the lat/long of the attacks and the type of event
country_map = niger_res[["latitude", 'longitude', 'event_type']]

country_map[['latitude', 'longitude']] = country_map[['latitude', 'longitude']].astype(float)

#see the data this time first 7 rows
country_map.head(7)


,latitude,longitude,event_type
0,13.3200,12.6100,Riots
1,13.5200,2.1200,Protests
2,13.6500,9.1833,Protests
3,13.6669,13.1198,Violence against civilians
4,13.1981,12.4835,Violence against civilians
5,13.3270,6.8968,Battles
6,13.6653,13.2733,Strategic developments


In [73]:
events_by_type = {} #make an empty datastructure (dictionary) to fill

#This loop goes through each row and counts the number of entries by group
for index, row in country_map.iterrows(): 
    if row["event_type"]  in events_by_type.keys():
        events_by_type[row["event_type"]] += 1 #if group is in the dictionary add 1 attack
    else:
        events_by_type[row["event_type"]] = 1 #add group name to dictionary if not in dictionary

events_by_type

{'Riots': 114,
 'Protests': 135,
 'Violence against civilians': 505,
 'Battles': 410,
 'Strategic developments': 164,
 'Explosions/Remote violence': 89}

In [72]:
events = set(country_map.event_type)
events

{'Battles',
 'Explosions/Remote violence',
 'Protests',
 'Riots',
 'Strategic developments',
 'Violence against civilians'}

In [48]:
#create pyproj transformer to convert form lat/long to web mercator
transformer = Transformer.from_crs('epsg:4326','epsg:3857')

In [74]:
map_dict = {} # empty dictionary to track events by lat long
nan_count = {} # some data doesn't have a lat/long so we need to know what we are losing


# Iterate through tables and associate group with lat/long
for idx, row in country_map.iterrows():    
    if row['event_type'] in map_dict.keys(): 
        if math.isnan(row["latitude"]):
            #This counts no data
            if row['event_type'] in nan_count.keys(): 
                nan_count[row['event_type']] += 1 
            else: 
                nan_count[row['event_type']] = 1
        else: 
            #This has to convert the lat/long to a mercator projection 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['event_type']].append([point[0],point[1]]) 
    #BOTH the if an else statement do the same thing but since it is a dictionary one needs to add the group name first
    else:  
        if math.isnan(row["latitude"]):
          nan_count[row['event_type']] = 1
        else: 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['event_type']] =[[point[0],point[1]]]
        
#This tells how many attacks we are losing
nan_count    

{}

In [78]:
# Create the bounding box for Niger

pts = [(23, -1), (12, 17)]
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)
    
# bbox

In [79]:
NPA_x = []
NPA_y = []
for pt in map_dict["Violence against civilians"]:
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])

In [80]:
#Plots the bounding box
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
# Places a circle for each converted lat/long attack 
p.circle(x = NPA_x, y = NPA_y, color= "firebrick")

#shows the plot
show(p)


In [ ]:
### PIKE Pike Sorry I didn't get this in sooner - I was hoping the 'productive
### Struggle' would help me to be able to use this on the final project... 
### There was a lot more to it than I thought at first.